In [ ]:
import numpy as np
import tensorflow as tf
import scipy.io
from keras import backend as K
from keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects
from scipy.special import jv

In [ ]:
# define the filename to save the weights of the model as a .mat
filename = 'models/oscillatory'

# define the interval of the domain for the visible dynamics
lower_limit = 0.1
upper_limit = 12

step_size = 0.001;

# define the approximate size the small offset parameter in the smooth ReLU
gamma = 1

# define the approximate size the feedback offset parameter in the feedback motif
beta_1_and_2 = 1

# define the number of hidden species in a single hidden layer
N = 6

# define the number of epochs to use in training the model
number_of_epochs = 2

In [ ]:
# define the custom activation function for the training with particular values of the relu_offset
def smooth_max_activation(x):
    return 0.5*(x + K.sqrt(K.square(x)+4*gamma))

get_custom_objects().update({'smooth_max_activation': Activation(smooth_max_activation)})

#define data
x1_train = np.arange(lower_limit, upper_limit, step_size, dtype="float32")
x2_train = np.arange(lower_limit, upper_limit, step_size, dtype="float32")

x1v, x2v = np.meshgrid(x1_train, x2_train, indexing='ij')
nx1 = len(x1_train)
nx2 = len(x2_train)

g_1 = 6+4*jv(0, 1.5*x1v)-x2v;
g_2 =  x1v - 4;

y1_train = (g_1 - beta_1_and_2)/x1v
y2_train = (g_2 - beta_1_and_2)/x2v

x_train = np.append(x1v.reshape(-1,1), x2v.reshape(-1,1),axis=1)
y_train = np.append(y1_train.reshape(-1,1), y2_train.reshape(-1,1),axis=1)


In [ ]:
# define a neural network model that corresponds to the asymptotic neural subsytem 
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(2,), name=''),
  tf.keras.layers.Dense(N, activation='smooth_max_activation'),
  tf.keras.layers.Dense(2, activation=None, use_bias=False),
])

# compile model with optimizer and choice of loss function
model.compile(optimizer='adam',
              loss='mse',
              metrics=['mse'])

In [ ]:
# train the neural network model to replicate x_train -> y_train
model.fit(x_train, y_train, epochs=number_of_epochs, validation_split = 0.1)

In [ ]:
# view the model structure, each 'Param' corresponds to at least one rate of reaction in the chemical system
model.summary()

# save the weights of this neural network for use in ODE simulations in MATLAB
first_layer_weights = model.layers[1].get_weights()[0]
first_layer_biases = model.layers[1].get_weights()[1]
output_layer_weights = model.layers[2].get_weights()[0]

In [ ]:
scipy.io.savemat(filename+'.mat', {'first_layer_weights':first_layer_weights, 
                                   'first_layer_biases':first_layer_biases, 
                                   'output_layer_weights':output_layer_weights,
                                   'gamma': gamma,
                                   'betas': beta_1_and_2,
                                  })